In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = irgan_config.epochs_d
epochs_g = irgan_config.epochs_g
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-dns-gen-Adam-G-SGD-D/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-dns-gen-Adam-G-SGD-D/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.Adam(G.parameters(), lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), lr = eta_D, weight_decay = weight_decay_d)

In [7]:
pretrained_model = torch.load("./pretrained_models/pretrained_model_dns.pkl",map_location=irgan_config.device)
G.load_state_dict(pretrained_model)

<All keys matched successfully>

In [ ]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.67386] [Time:15.26133] 
	[Discriminator][Epochs 2/100] [D epoch loss: 0.63978] [Time:14.57093] 
	[Discriminator][Epochs 3/100] [D epoch loss: 0.61148] [Time:14.24633] 
	[Discriminator][Epochs 4/100] [D epoch loss: 0.58767] [Time:14.12948] 
	[Discriminator][Epochs 5/100] [D epoch loss: 0.56737] [Time:14.14009] 
	[Discriminator][Epochs 6/100] [D epoch loss: 0.54986] [Time:14.52194] 
	[Discriminator][Epochs 7/100] [D epoch loss: 0.53457] [Time:14.28937] 
	[Discriminator][Epochs 8/100] [D epoch loss: 0.52109] [Time:13.65670] 
	[Discriminator][Epochs 9/100] [D epoch loss: 0.50910] [Time:14.46498] 
	[Discriminator][Epochs 10/100] [D epoch loss: 0.49836] [Time:14.59846] 
	[Discriminator][Epochs 11/100] [D epoch loss: 0.48856] [Time:14.44253] 
	[Discriminator][Epochs 12/100] [D epoch loss: 0.47974] [Time:13.14091] 
	[Discriminator][Epochs 13/100] [D epoch loss: 0.47170] [Time:13.67561] 
	[Discriminator][Epochs 14/100] [D epoch loss:

	[Generator][Epochs 15/50] [G epoch loss: 0.78758] [Time:0.04232]
	[Generator][Epochs 16/50] [G epoch loss: 0.74973] [Time:0.04187]
	[Generator][Epochs 17/50] [G epoch loss: 0.69407] [Time:0.04188]
	[Generator][Epochs 18/50] [G epoch loss: 0.69747] [Time:0.04182]
	[Generator][Epochs 19/50] [G epoch loss: 0.64205] [Time:0.04214]
	[Generator][Epochs 20/50] [G epoch loss: 0.52472] [Time:0.06324]
	[Generator][Epochs 21/50] [G epoch loss: 0.46386] [Time:0.05657]
	[Generator][Epochs 22/50] [G epoch loss: 0.40582] [Time:0.05670]
	[Generator][Epochs 23/50] [G epoch loss: 0.38188] [Time:0.04372]
	[Generator][Epochs 24/50] [G epoch loss: 0.35017] [Time:0.04226]
	[Generator][Epochs 25/50] [G epoch loss: 0.32556] [Time:0.04342]
	[Generator][Epochs 26/50] [G epoch loss: 0.31361] [Time:0.04205]
	[Generator][Epochs 27/50] [G epoch loss: 0.28801] [Time:0.05953]
	[Generator][Epochs 28/50] [G epoch loss: 0.27683] [Time:0.04246]
	[Generator][Epochs 29/50] [G epoch loss: 0.26290] [Time:0.04167]
	[Generato

	[Discriminator][Epochs 81/100] [D epoch loss: 0.24448] [Time:14.59851] 
	[Discriminator][Epochs 82/100] [D epoch loss: 0.24410] [Time:13.52570] 
	[Discriminator][Epochs 83/100] [D epoch loss: 0.24372] [Time:14.44664] 
	[Discriminator][Epochs 84/100] [D epoch loss: 0.24336] [Time:14.55503] 
	[Discriminator][Epochs 85/100] [D epoch loss: 0.24299] [Time:14.46016] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.24265] [Time:15.19956] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.24229] [Time:14.72675] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.24195] [Time:14.67794] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.24161] [Time:14.05946] 
	[Discriminator][Epochs 90/100] [D epoch loss: 0.24127] [Time:12.04282] 
	[Discriminator][Epochs 91/100] [D epoch loss: 0.24094] [Time:13.13753] 
	[Discriminator][Epochs 92/100] [D epoch loss: 0.24062] [Time:14.20543] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.24030] [Time:14.86577] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.23

	[Discriminator][Epochs 49/100] [D epoch loss: 0.26438] [Time:14.44819] 
	[Discriminator][Epochs 50/100] [D epoch loss: 0.26342] [Time:15.86516] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.26249] [Time:15.57023] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.26160] [Time:14.96625] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.26073] [Time:15.09589] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.25990] [Time:14.43543] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.25910] [Time:14.12369] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.25831] [Time:15.15506] 
	[Discriminator][Epochs 57/100] [D epoch loss: 0.25756] [Time:14.87403] 
	[Discriminator][Epochs 58/100] [D epoch loss: 0.25683] [Time:14.78559] 
	[Discriminator][Epochs 59/100] [D epoch loss: 0.25612] [Time:14.13486] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.25543] [Time:15.07314] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.25476] [Time:15.26243] 
	[Discriminator][Epochs 62/100] [D epoch loss: 0.25

	[Discriminator][Epochs 17/100] [D epoch loss: 0.34239] [Time:14.63449] 
	[Discriminator][Epochs 18/100] [D epoch loss: 0.33615] [Time:13.24587] 
	[Discriminator][Epochs 19/100] [D epoch loss: 0.33049] [Time:13.73453] 
	[Discriminator][Epochs 20/100] [D epoch loss: 0.32531] [Time:14.42650] 
	[Discriminator][Epochs 21/100] [D epoch loss: 0.32057] [Time:14.80064] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.31620] [Time:14.59844] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.31218] [Time:14.66037] 
	[Discriminator][Epochs 24/100] [D epoch loss: 0.30845] [Time:14.34690] 
	[Discriminator][Epochs 25/100] [D epoch loss: 0.30499] [Time:14.49336] 
	[Discriminator][Epochs 26/100] [D epoch loss: 0.30178] [Time:14.75178] 
	[Discriminator][Epochs 27/100] [D epoch loss: 0.29877] [Time:13.75529] 
	[Discriminator][Epochs 28/100] [D epoch loss: 0.29597] [Time:12.56476] 
	[Discriminator][Epochs 29/100] [D epoch loss: 0.29333] [Time:14.70461] 
	[Discriminator][Epochs 30/100] [D epoch loss: 0.29

...

	[Discriminator][Epochs 45/100] [D epoch loss: 0.20523] [Time:14.64905] 
	[Discriminator][Epochs 46/100] [D epoch loss: 0.20509] [Time:15.04963] 
	[Discriminator][Epochs 47/100] [D epoch loss: 0.20495] [Time:14.00291] 
	[Discriminator][Epochs 48/100] [D epoch loss: 0.20480] [Time:13.08234] 
	[Discriminator][Epochs 49/100] [D epoch loss: 0.20466] [Time:14.06770] 
	[Discriminator][Epochs 50/100] [D epoch loss: 0.20453] [Time:13.54814] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.20439] [Time:15.02316] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.20425] [Time:13.76951] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.20411] [Time:13.48185] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.20398] [Time:14.35239] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.20384] [Time:14.03812] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.20371] [Time:14.46707] 
	[Discriminator][Epochs 57/100] [D epoch loss: 0.20357] [Time:14.06206] 
	[Discriminator][Epochs 58/100] [D epoch loss: 0.20

	[Discriminator][Epochs 12/100] [D epoch loss: 0.19741] [Time:14.22539] 
	[Discriminator][Epochs 13/100] [D epoch loss: 0.19730] [Time:14.60550] 
	[Discriminator][Epochs 14/100] [D epoch loss: 0.19719] [Time:15.34482] 
	[Discriminator][Epochs 15/100] [D epoch loss: 0.19708] [Time:15.37610] 
	[Discriminator][Epochs 16/100] [D epoch loss: 0.19697] [Time:14.11478] 
	[Discriminator][Epochs 17/100] [D epoch loss: 0.19686] [Time:13.95314] 
	[Discriminator][Epochs 18/100] [D epoch loss: 0.19675] [Time:15.24307] 
	[Discriminator][Epochs 19/100] [D epoch loss: 0.19664] [Time:15.29174] 
	[Discriminator][Epochs 20/100] [D epoch loss: 0.19654] [Time:15.32662] 
	[Discriminator][Epochs 21/100] [D epoch loss: 0.19643] [Time:13.05143] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.19633] [Time:13.67428] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.19622] [Time:15.08192] 
	[Discriminator][Epochs 24/100] [D epoch loss: 0.19612] [Time:15.29821] 
	[Discriminator][Epochs 25/100] [D epoch loss: 0.19

	[Generator][Epochs 27/50] [G epoch loss: -0.04212] [Time:0.04259]
	[Generator][Epochs 28/50] [G epoch loss: -0.04480] [Time:0.04215]
	[Generator][Epochs 29/50] [G epoch loss: -0.05403] [Time:0.04199]
	[Generator][Epochs 30/50] [G epoch loss: -0.07013] [Time:0.04271]
	[Generator][Epochs 31/50] [G epoch loss: -0.17793] [Time:0.04248]
	[Generator][Epochs 32/50] [G epoch loss: -0.26656] [Time:0.04209]
	[Generator][Epochs 33/50] [G epoch loss: -0.43611] [Time:0.04244]
	[Generator][Epochs 34/50] [G epoch loss: -0.46831] [Time:0.04241]
	[Generator][Epochs 35/50] [G epoch loss: -0.39507] [Time:0.04218]
	[Generator][Epochs 36/50] [G epoch loss: -0.29211] [Time:0.04191]
	[Generator][Epochs 37/50] [G epoch loss: -0.25415] [Time:0.04262]
	[Generator][Epochs 38/50] [G epoch loss: -0.19381] [Time:0.04221]
	[Generator][Epochs 39/50] [G epoch loss: -0.15277] [Time:0.04240]
	[Generator][Epochs 40/50] [G epoch loss: -0.15680] [Time:0.04249]
	[Generator][Epochs 41/50] [G epoch loss: -0.20159] [Time:0.04

	[Discriminator][Epochs 92/100] [D epoch loss: 0.19856] [Time:15.19159] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.19834] [Time:14.91279] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.19812] [Time:14.70796] 
	[Discriminator][Epochs 95/100] [D epoch loss: 0.19791] [Time:14.52633] 
	[Discriminator][Epochs 96/100] [D epoch loss: 0.19770] [Time:13.41273] 
	[Discriminator][Epochs 97/100] [D epoch loss: 0.19749] [Time:13.75794] 
	[Discriminator][Epochs 98/100] [D epoch loss: 0.19729] [Time:14.87780] 
	[Discriminator][Epochs 99/100] [D epoch loss: 0.19708] [Time:14.61042] 
	[Discriminator][Epochs 100/100] [D epoch loss: 0.19688] [Time:12.50015] 
	[Generator][Epochs 1/50] [G epoch loss: -0.28058] [Time:0.05926]
	[Generator][Epochs 2/50] [G epoch loss: -0.28454] [Time:0.05407]
	[Generator][Epochs 3/50] [G epoch loss: -0.21492] [Time:0.06044]
	[Generator][Epochs 4/50] [G epoch loss: -0.22452] [Time:0.06860]
	[Generator][Epochs 5/50] [G epoch loss: -0.21375] [Time:0.04566]
	[Generator]

	[Discriminator][Epochs 59/100] [D epoch loss: 0.21763] [Time:14.55149] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.21703] [Time:14.05647] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.21644] [Time:14.95789] 
	[Discriminator][Epochs 62/100] [D epoch loss: 0.21588] [Time:14.71433] 
	[Discriminator][Epochs 63/100] [D epoch loss: 0.21532] [Time:14.93104] 
	[Discriminator][Epochs 64/100] [D epoch loss: 0.21478] [Time:14.26868] 
	[Discriminator][Epochs 65/100] [D epoch loss: 0.21426] [Time:13.42135] 
	[Discriminator][Epochs 66/100] [D epoch loss: 0.21375] [Time:14.75884] 
	[Discriminator][Epochs 67/100] [D epoch loss: 0.21325] [Time:13.59977] 
	[Discriminator][Epochs 68/100] [D epoch loss: 0.21276] [Time:13.16427] 
	[Discriminator][Epochs 69/100] [D epoch loss: 0.21229] [Time:13.45420] 
	[Discriminator][Epochs 70/100] [D epoch loss: 0.21183] [Time:14.01803] 
	[Discriminator][Epochs 71/100] [D epoch loss: 0.21138] [Time:14.25947] 
	[Discriminator][Epochs 72/100] [D epoch loss: 0.21